In [74]:
### Read the Household Data ########################

In [75]:
import pandas as pd
import numpy as np
import os
import time
DATA_DIR_H = "/Users/abhayranjan/scripts/GA/predicting-poverty"

data_h_paths = {'A': {'train': os.path.join(DATA_DIR_H, 'A', 'A_hhold_train.csv'),
                    'test' : os.path.join(DATA_DIR_H, 'A', 'A_hhold_test.csv')},
              'B': {'train': os.path.join(DATA_DIR_H, 'B', 'B_hhold_train.csv'),
                    'test' : os.path.join(DATA_DIR_H, 'B', 'B_hhold_test.csv')},
              'C' :{'train': os.path.join(DATA_DIR_H, 'C', 'C_hhold_train.csv'),
                    'test' : os.path.join(DATA_DIR_H, 'C', 'C_hhold_test.csv')}}

In [76]:
a_h_train = pd.read_csv(data_h_paths['A']['train'],index_col='id')
a_h_test  = pd.read_csv(data_h_paths['A']['test'],index_col='id')
b_h_train = pd.read_csv(data_h_paths['B']['train'],index_col='id')
b_h_test  = pd.read_csv(data_h_paths['B']['test'],index_col='id')
c_h_train = pd.read_csv(data_h_paths['C']['train'],index_col='id')
c_h_test  = pd.read_csv(data_h_paths['C']['test'],index_col='id')

In [77]:
print(a_h_train.poor.value_counts())
print(b_h_train.poor.value_counts())
print(c_h_train.poor.value_counts())

False    4500
True     3703
Name: poor, dtype: int64
False    3004
True      251
Name: poor, dtype: int64
False    5496
True      973
Name: poor, dtype: int64


In [78]:
### Read the Individual Dataset #####################################

In [79]:
DATA_DIR_I = "/Users/abhayranjan/scripts/GA/predicting-poverty"

data_i_paths = {'A': {'train': os.path.join(DATA_DIR_I, 'A', 'A_indiv_train.csv'),
                    'test' : os.path.join(DATA_DIR_I, 'A', 'A_indiv_test.csv')},
              'B': {'train': os.path.join(DATA_DIR_I, 'B', 'B_indiv_train.csv'),
                    'test' : os.path.join(DATA_DIR_I, 'B', 'B_indiv_test.csv')},
              'C' :{'train': os.path.join(DATA_DIR_I, 'C', 'C_indiv_train.csv'),
                    'test' : os.path.join(DATA_DIR_I, 'C', 'C_indiv_test.csv')}}

In [80]:
a_i_train = pd.read_csv(data_i_paths['A']['train'],index_col='id')
a_i_test  = pd.read_csv(data_i_paths['A']['test'],index_col='id')
b_i_train = pd.read_csv(data_i_paths['B']['train'],index_col='id')
b_i_test  = pd.read_csv(data_i_paths['B']['test'],index_col='id')
c_i_train = pd.read_csv(data_i_paths['C']['train'],index_col='id')
c_i_test  = pd.read_csv(data_i_paths['C']['test'],index_col='id')

In [81]:
print(a_i_train.poor.value_counts())
print(b_i_train.poor.value_counts())
print(c_i_train.poor.value_counts())

True     19684
False    17876
Name: poor, dtype: int64
False    18375
True      1877
Name: poor, dtype: int64
False    22868
True      7045
Name: poor, dtype: int64


In [82]:
def treat_missing_values(dataframe):
    numeric = dataframe.select_dtypes(include=['int64','float64'])
    objectt = dataframe.select_dtypes(include=['object'])
    for col in numeric.columns:
        dataframe[str(col)].fillna(0,inplace=True)
    for col in objectt.columns:
        dataframe[str(col)].fillna(dataframe[str(col)].mode(),inplace=True)
    return dataframe

In [83]:
def normalize_dataset(dataframe,numeric_only=True):
    numeric = dataframe.select_dtypes(include=['int64','float64'])
    dataframe[numeric.columns] = numeric -numeric.mean()/numeric.std()
    return dataframe

In [84]:
# Feature Dummification
def create_feature_dummies(dataframe):
    dataframe = pd.get_dummies(dataframe)
    return dataframe

In [85]:
def pre_process(dataframe,enforce_cols=None):
    if enforce_cols is not None:
        to_drop = np.setdiff1d(dataframe.columns,enforce_cols)
        to_add = np.setdiff1d(enforce_cols,dataframe.columns)
        dataframe.drop(to_drop,axis=1,inplace=True)
        dataframe = dataframe.assign(**{c: 0 for c in to_add})
    return dataframe

In [86]:
def aggregate_data(dataframe,id,country):
    flattened_dataframe = pd.DataFrame()
    #print(type(dataframe))
    #print("Index:",id)
    if (country == "A"):
        dataframe = dataframe.drop(['country_A','iid'], axis=1)
    elif (country =="B"):
        dataframe = dataframe.drop(['country_B','iid'], axis=1)
    else:
        dataframe = dataframe.drop(['country_C','iid'], axis=1)
    #print((dataframe.dtypes).unique())
    #print("Length of all the columns:",len(dataframe.columns.values))
    numeric = dataframe.select_dtypes(include=['uint8'])
    numericcols = numeric.columns
    #print("Length of Numeric columns:",len(numericcols))
    numericcolsdict = {col:'sum' for col in numericcols.tolist()}
    #print(numericcolsdict)
    uint_dataframe = dataframe[dataframe.index == id].groupby([dataframe.index]).agg(numericcolsdict)
    #print(uint_dataframe)
    floatt = dataframe.select_dtypes(include=['float'])
    floatcols = floatt.columns
    #print("Length of Float columns:",len(floatcols))
    floatcolslist = {col:'sum' for col in floatcols.tolist()}
    float_dataframe = dataframe[dataframe.index == id].groupby([dataframe.index]).agg(floatcolslist)
    #print(float_dataframe)
    booll = dataframe.select_dtypes(include=['bool'])
    boolcols = booll.columns
    #print("Length of Float columns:",len(boolcols))
    boolcolslist = {col:'mean' for col in boolcols.tolist()}
    bool_dataframe = dataframe[dataframe.index == id].groupby([dataframe.index]).agg(boolcolslist)
    #print(bool_dataframe)
    flattened_dataframe = pd.concat([flattened_dataframe,uint_dataframe,float_dataframe,bool_dataframe], axis=1)
    return flattened_dataframe

In [87]:
a_h_train = treat_missing_values(a_h_train)
a_i_train = treat_missing_values(a_i_train)

In [88]:
a_h_train = normalize_dataset(a_h_train)
a_i_train = normalize_dataset(a_i_train)

In [89]:
a_h_train = create_feature_dummies(a_h_train)
a_i_train = create_feature_dummies(a_i_train)

In [90]:
aggregated_df = pd.DataFrame()

In [91]:
#a_i_train.loc[59007]

In [92]:
from multiprocessing.pool import ThreadPool
pool = ThreadPool(processes=14)

final_train_df_a = pd.DataFrame()
results = []
start_time = time.time()
print("Time before starting the agregation operation for training set A:",start_time)
for index in set(a_i_train.index.values):
    async_ret = pool.apply_async(aggregate_data, (a_i_train[a_i_train.index == index],index,"A")) # tuple of args for foo
    results.append(async_ret)

print('Scheduling complete... Waiting for results.')

for res in results:
    final_train_df_a = final_train_df_a.append(res.get())

print("Time after finishing the agregation operation for training set A:",time.time()-start_time)
print('Run complete.')

Time before starting the agregation operation for training set A: 1518964510.942912
Scheduling complete... Waiting for results.
Time after finishing the agregation operation for training set A: 1369.082855939865
Run complete.


In [93]:
#final_train_df_a.head(n=50)

In [94]:
import time
from multiprocessing.pool import ThreadPool
pool = ThreadPool(processes=4)

def aggregate_data_test(x):
    time.sleep(np.random.randint(2,5))
    return x**2

results = []

for i in range(10):
    results.append(pool.apply_async(aggregate_data_test, (i,))) # tuple of args for foo

for res in results:
    print(res.get())

print('Done')

0
1
4
9
16
25
36
49
64
81
Done


In [95]:
# Remove duplicate rows from the individual falttened dataframe and concat it with household dataframe
final_train_df_a.drop_duplicates(inplace=True)
a_final_train_dataframe = pd.merge(final_train_df_a,a_h_train,left_index=True,right_index=True)
a_final_train_dataframe = a_final_train_dataframe.drop(['poor_y'],axis=1)
a_final_train_dataframe.rename(columns ={'poor_x':'poor'},inplace=True)
a_final_train_dataframe.head(n=5)

,HeUgMnzF_BNCcM,HeUgMnzF_HUpWg,HeUgMnzF_JMXQx,HeUgMnzF_PAVsH,HeUgMnzF_SJPkb,HeUgMnzF_SlRmt,HeUgMnzF_TRFeI,HeUgMnzF_XJgvq,HeUgMnzF_XJsPz,HeUgMnzF_YsSBt,HeUgMnzF_ayXFR,HeUgMnzF_dHZCo,HeUgMnzF_duBym,HeUgMnzF_fmdsF,HeUgMnzF_jnwBm,HeUgMnzF_kuFXw,HeUgMnzF_lBMrM,HeUgMnzF_nIete,HeUgMnzF_oGavK,HeUgMnzF_slvTJ,HeUgMnzF_tMiQp,HeUgMnzF_vhanw,HeUgMnzF_wWIzo,HeUgMnzF_xnnDH,HeUgMnzF_xsVYp,CaukPfUC_kzSFB,CaukPfUC_mOlYV,CaukPfUC_yAyAe,MzEtIdUF_FRcdT,MzEtIdUF_UFoKR,MzEtIdUF_axSTs,gtnNTNam_CXizI,gtnNTNam_DQhEE,gtnNTNam_DaETh,gtnNTNam_GotAd,gtnNTNam_HIvIU,gtnNTNam_JyIRx,gtnNTNam_LvUxT,gtnNTNam_SSvEP,gtnNTNam_TKOAj,gtnNTNam_UklJv,gtnNTNam_YsahA,gtnNTNam_hCIpw,gtnNTNam_lzzev,gtnNTNam_mlNXN,gtnNTNam_msICg,gtnNTNam_tOfmA,SWoXNmPc_CNSXC,SWoXNmPc_GNSwM,SWoXNmPc_NDnCs,SWoXNmPc_OnQJl,SWoXNmPc_jPwzS,SWoXNmPc_onRNG,eXbOkwhI_AvBOo,eXbOkwhI_MwnxJ,eXbOkwhI_YXCNt,XONDGWjH_AOSWk,XONDGWjH_BqqGq,XONDGWjH_LUHXb,XONDGWjH_QyhRH,XONDGWjH_WKBXv,XONDGWjH_XYcCK,XONDGWjH_XvoCa,XONDGWjH_YslGF,XONDGWjH_aPsLh,XONDGWjH_ccbZA,XONDGWjH_fOUHD,XONDGWjH_nLEWH,XONDGWjH_oArAw,XONDGWjH_pWLuE,XONDGWjH_vkRKJ,KsFoQcUV_HgfUG,KsFoQcUV_JepVl,KsFoQcUV_OQaAJ,KsFoQcUV_dPGpn,KsFoQcUV_kpkiH,qYRZCuJD_fohru,qYRZCuJD_oEref,qYRZCuJD_xMiWa,FPQrjGnS_BRiui,FPQrjGnS_HRGCq,FPQrjGnS_bJTYb,FPQrjGnS_jAYfu,FPQrjGnS_rwCRh,FPQrjGnS_scxJu,FPQrjGnS_tqINY,FPQrjGnS_yomtK,hOamrctW_AwvHz,hOamrctW_IxnPt,hOamrctW_OMzWB,hOamrctW_Whopv,hOamrctW_YoNmC,hOamrctW_cHNSE,hOamrctW_cpPKr,hOamrctW_iWGMu,hOamrctW_jTlga,hOamrctW_kNZZn,hOamrctW_lwhhy,hOamrctW_oWkda,hOamrctW_oyoSU,hOamrctW_rrHdI,hOamrctW_zCwHm,XacGrSou_AYcgs,XacGrSou_CSuYG,XacGrSou_DYDEc,XacGrSou_DgtXD,XacGrSou_EaHvf,XacGrSou_GmSKW,XacGrSou_HHynv,XacGrSou_HREad,XacGrSou_MEcMe,XacGrSou_OZZvs,XacGrSou_QdBZK,XacGrSou_RGHDu,XacGrSou_RHfBO,XacGrSou_VzUws,XacGrSou_YEngm,XacGrSou_aJiqV,XacGrSou_gjYlb,XacGrSou_nwpOp,XacGrSou_pRitH,XacGrSou_tymHY,XacGrSou_uHQhm,XacGrSou_uhOlG,XacGrSou_yhUHu,XacGrSou_zfTDU,XacGrSou_zwajg,UsmeXdIS_IZbuU,UsmeXdIS_kzJXk,UsmeXdIS_olfwp,UsmeXdIS_pdgUV,UsmeXdIS_qIbMY,UsmeXdIS_sDvAm,UsmeXdIS_xrEKh,igHwZsYz_BQEnF,igHwZsYz_GJlUg,igHwZsYz_Rjkzz,igHwZsYz_VGNER,igHwZsYz_XKQWl,igHwZsYz_YwljV,igHwZsYz_bszTA,igHwZsYz_flBEG,igHwZsYz_iyqBV,igHwZsYz_kFEpL,igHwZsYz_lcEtN,igHwZsYz_xBZrP,cxWuAOZv_QkRds,cxWuAOZv_sIcXU,cxWuAOZv_veBMo,AQpdiRUz_QjTos,AQpdiRUz_SowpV,AQpdiRUz_nUKzL,AoLwmlEH_GdxzZ,AoLwmlEH_OeQKE,AoLwmlEH_TGbFh,AoLwmlEH_XNPgB,AoLwmlEH_vSaJn,AoLwmlEH_veIDf,nLUXHpZr_Asxgp,nLUXHpZr_CneHb,nLUXHpZr_FcQGH,nLUXHpZr_HAldK,nLUXHpZr_JPCna,nLUXHpZr_MxNAc,nLUXHpZr_SuzRU,nLUXHpZr_vvXmD,CRLlSiFu_PaHYu,CRLlSiFu_PmhpI,CRLlSiFu_SfJhI,CRLlSiFu_SjaWF,CRLlSiFu_TUafC,CRLlSiFu_XXwfL,CRLlSiFu_bazjA,CRLlSiFu_cqiXu,CRLlSiFu_dpMMl,CRLlSiFu_fEUOi,CRLlSiFu_jECfI,CRLlSiFu_meQRz,CRLlSiFu_nPyMt,CRLlSiFu_nbJOd,CRLlSiFu_ndArQ,CRLlSiFu_pkNwY,CRLlSiFu_qVwNL,CRLlSiFu_wnmbT,CRLlSiFu_zTqjB,jYpOAjPW_BNylo,jYpOAjPW_CXjLj,jYpOAjPW_KOjYm,NAxEQZVi_BgmGE,NAxEQZVi_GIApU,NAxEQZVi_GkrMH,NAxEQZVi_MZyJF,NAxEQZVi_PwkMV,NAxEQZVi_QSfdN,NAxEQZVi_Rihyc,NAxEQZVi_bipfP,QvgxCmCV_AyuSE,QvgxCmCV_FeEPn,QvgxCmCV_GxyHv,QvgxCmCV_PrZhn,QvgxCmCV_ZApCl,QvgxCmCV_hCKQi,QvgxCmCV_nGyVK,QvgxCmCV_rGwNF,kvMGuSDN_Qydia,kvMGuSDN_vtkRP,AjYsrafY_NTOkI,AjYsrafY_OLBHI,AjYsrafY_kVYrO,AjYsrafY_lCwhY,AjYsrafY_sqnlK,GNaMafZC_ETgtR,GNaMafZC_sitaC,GNaMafZC_tLMKW,GNaMafZC_yDvyE,zTLPJYTw_VneGw,zTLPJYTw_rXEFU,zTLPJYTw_zncPX,QKzxTGus_EAWFH,QKzxTGus_WNISg,QKzxTGus_aKoLM,SGeOiUlZ_DGyQh,SGeOiUlZ_OlSuJ,SGeOiUlZ_OoqEw,SGeOiUlZ_RuCZA,SGeOiUlZ_SWhXf,SGeOiUlZ_UCsCT,SGeOiUlZ_cEcbt,SGeOiUlZ_sSQyF,SGeOiUlZ_uJdwX,SGeOiUlZ_xjHpn,SGeOiUlZ_yJFxH,MUrHEJeh_QBrMF,MUrHEJeh_mEGPl,MUrHEJeh_qmOVd,XBldkztv_XQevi,XBldkztv_tbsMf,tbgZsPXD_RPBUw,tbgZsPXD_yOwsR,qqVibbSA_IqFnZ,qqVibbSA_Jarbl,qqVibbSA_QQdHS,qqVibbSA_SPVgF,qqVibbSA_ZIrnY,qqVibbSA_exnST,qqVibbSA_sMBUT,MgCoFhXK_dAmhs,MgCoFhXK_gCSRj,MgCoFhXK_uEstx,rFpoTXAq_ESfgE,rFpoTXAq_Hikoa,rFpoTXAq_okwnE,RXcLsVAQ_AkQtO,RXcLsVAQ_Coacj,RXcLsVAQ_JuVFO,RXcLsVAQ_LxRxW,RXcLsVAQ_OkXob,RXcLsVAQ_dDnIb,RXcLsVAQ_whJCg,RXcLsVAQ_zQvdC,rQWIpTiG_jVHyH,rQWIpTiG_rkLqZ,rQWIpTiG_xUYIC,XizJGmbu_FUUXv,XizJGmbu_GtHel,XizJGmbu_juMSt,xqUooaNJ_

In [96]:
# Prepare the test dataset for prediction

In [97]:
a_h_test = treat_missing_values(a_h_test)
a_i_test = treat_missing_values(a_i_test)

In [98]:
a_h_test = normalize_dataset(a_h_test)
a_i_test = normalize_dataset(a_i_test)

In [99]:
a_h_test = create_feature_dummies(a_h_test)
a_i_test = create_feature_dummies(a_i_test)

In [100]:
from multiprocessing.pool import ThreadPool
pool = ThreadPool(processes=14)

final_test_df_a = pd.DataFrame()
results = []
start_time = time.time()
print("Time before starting the agregation operation for testing set A:",start_time)
for index in set(a_i_test.index.values):
    async_ret = pool.apply_async(aggregate_data, (a_i_test[a_i_test.index == index],index,"A")) # tuple of args for foo
    results.append(async_ret)

print('Scheduling complete... Waiting for results.')

for res in results:
    final_test_df_a = final_test_df_a.append(res.get())

print("Time after finishing the agregation operation for testing set A:",time.time()-start_time)
print('Run complete.')

Time before starting the agregation operation for testing set A: 1518965890.990062
Scheduling complete... Waiting for results.
Time after finishing the agregation operation for testing set A: 656.8166737556458
Run complete.


In [101]:
final_test_df_a.drop_duplicates(inplace=True)
a_final_test_dataframe = pd.merge(final_test_df_a,a_h_test,left_index=True,right_index=True)
a_final_test_dataframe.head(n=5)
a_final_test_dataframe.to_csv('test1233.csv')

In [102]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 2000

In [103]:
#a_final_test_dataframe.columns.values
a_final_test_dataframe.head(n=1)
#a_final_test_dataframe.drop(['poor'],axis=1,inplace=True)
print(len(a_final_test_dataframe.columns.values))

1123


In [104]:
print(set(a_final_train_dataframe.columns.values) - set(a_final_test_dataframe.columns.values))
#a_final_test_dataframe = pre_process(a_final_test_dataframe,enforce_cols=a_final_train_dataframe.columns)
a_final_test_dataframe = pre_process(a_final_test_dataframe,enforce_cols=a_final_train_dataframe.columns)
print(len(a_final_test_dataframe.columns.values))
print(len(a_final_train_dataframe.columns.values))

{'tHFrzjai_xBZqe', 'HeUgMnzF_JMXQx', 'UjuNwfjv_ZBMyY', 'NAxEQZVi_BgmGE', 'LjvKYNON_tFrTi', 'tHFrzjai_MkATH', 'nKoaotpH_HQYUq', 'HeUgMnzF_jnwBm', 'UjuNwfjv_ujWYB', 'nSzbETYS_yTPHV', 'UjuNwfjv_ACFXG', 'poor', 'qqVibbSA_sMBUT'}
1133
1133


In [105]:
# Initializing the Predictor and Target Variables for Modelling

In [106]:
from sklearn.model_selection import train_test_split
aX_train = a_final_train_dataframe.drop('poor',axis=1)
ay_train = a_final_train_dataframe['poor']
#aX_train1,aX_test1,ay_train1,ay_test1 = train_test_split(aX_train,ay_train,test_size=0.20,random_state=243)

In [107]:
from xgboost import XGBClassifier

model_a = XGBClassifier()
#model_a.fit(aX_train1,ay_train1)
model_a.fit(aX_train,ay_train)
a_final_test_dataframe = a_final_test_dataframe[aX_train.columns]
print(set(a_final_test_dataframe.columns.values) - set(aX_train.columns.values))
ay_pred = model_a.predict_proba(a_final_test_dataframe)
ay_pred

set()


array([[  1.79510713e-02,   9.82048929e-01],
       [  1.49960697e-01,   8.50039303e-01],
       [  9.30863380e-01,   6.91366196e-02],
       ..., 
       [  9.92234886e-01,   7.76510919e-03],
       [  9.99627173e-01,   3.72832408e-04],
       [  3.84841561e-01,   6.15158439e-01]], dtype=float32)

In [108]:
# Cleaning,Formatting and modelling for Country B

In [109]:
b_h_train = treat_missing_values(b_h_train)
b_i_train = treat_missing_values(b_i_train)

In [110]:
b_h_train = normalize_dataset(b_h_train)
b_i_train = normalize_dataset(b_i_train)

In [111]:
b_h_train = create_feature_dummies(b_h_train)
b_i_train = create_feature_dummies(b_i_train)

In [112]:
from multiprocessing.pool import ThreadPool
pool = ThreadPool(processes=14)

final_train_df_b = pd.DataFrame()
results = []
start_time = time.time()
print("Time before starting the agregation operation for training set B:",start_time)
for index in set(b_i_train.index.values):
    async_ret = pool.apply_async(aggregate_data, (b_i_train[b_i_train.index == index],index,"B")) # tuple of args for foo
    results.append(async_ret)

print('Scheduling complete... Waiting for results.')

for res in results:
    final_train_df_b = final_train_df_b.append(res.get())

print("Time after finishing the agregation operation for training set B:",time.time()-start_time)
print('Run complete.')

Time before starting the agregation operation for training set B: 1518966566.7376199
Scheduling complete... Waiting for results.
Time after finishing the agregation operation for training set B: 3089.7066462039948
Run complete.


In [113]:
# Remove duplicate rows from the individual falttened dataframe and concat it with household dataframe
final_train_df_b.drop_duplicates(inplace=True)
b_final_train_dataframe = pd.merge(final_train_df_b,b_h_train,left_index=True,right_index=True)
b_final_train_dataframe = b_final_train_dataframe.drop(['poor_y'],axis=1)
b_final_train_dataframe.rename(columns ={'poor_x':'poor'},inplace=True)
b_final_train_dataframe.head(n=5)

MmKGUOrD_ZZKZW  MmKGUOrD_dyqxw  MmKGUOrD_eMhLf  MmKGUOrD_vWHEF  \
id                                                                      
24576               8               0               0               4   
24577               5               0               1               0   
16386               5               0               1               0   
98310               7               0               0               2   
73735               4               0               0               0   

       splgBZfe_CLRvF  splgBZfe_JSjRa  splgBZfe_RAlRo  splgBZfe_UwJQF  \
id                                                                      
24576               9               0               0               0   
24577               5               0               0               0   
16386               6               0               0               0   
98310               7               0               0               0   
73735               3               0               0               0   

       splgBZfe_ZDeRX  splgBZfe_bHplF  splgBZfe_iEbqA  splgBZfe_jbrpw  \
id                                                                      
24576               0               3               0               0   
24577               0               1               0               0   
16386               0               0               0               0   
98310               0               2               0               0   
73735               0               0               0               0   

       splgBZfe_qLcdo  splgBZfe_raKBO  splgBZfe_tIZVV  splgBZfe_tpCfK  \
id                                                                      
24576               0               0               0               0   
24577               0               0               0               0   
16386               0               0               0               0   
98310               0               0               0               0   
73735               0               0               0               0   

       splgBZfe_uCnhp  splgBZfe_xiPzN  JTCCBpjX_QEcpz  JTCCBpjX_VhFZR  \
id                                                                      
24576               0               0               6               0   
24577               0               0               5               0   
16386               0               0               2               0   
98310               0               0               4               0   
73735               1               0               2               0   

       JTCCBpjX_kCoGg  akGaJaYF_RVWQa  akGaJaYF_pCUxR  akGaJaYF_wmLgk  \
id                                                                      
24576               6               0               6               6   
24577               1               0               1               5   
16386               4               0               4               2   
98310               5               0               5               4   
73735               2               0               2               2   

       eYoMkEyN_VsLed  eYoMkEyN_cRkfb  eYoMkEyN_rSPYW  bmhTgzYu_LRmij  \
id                                                                      
24576               7               5               0               4   
24577               5               1               0               3   
16386               2               4               0               5   
98310               5               4               0               6   
73735               2               2               0               1   

       bmhTgzYu_lfpaq  GjxOmHgp_KIyHi  GjxOmHgp_asPdv  GjxOmHgp_mYaHJ  \
id                                                                      
24576               8               0               0               0   
24577               3               0               0               0   
16386               1               0               0               0   
98310               3               0               0     

In [114]:
from sklearn.utils import resample

b_minority = b_final_train_dataframe[b_final_train_dataframe.poor == True]
b_majority = b_final_train_dataframe[b_final_train_dataframe.poor == False]
#Upsample minority class
b_minority_upsampled = resample(b_minority,replace=True,n_samples=3004,random_state=123)
b_final_train_dataframe = pd.concat([b_majority,b_minority_upsampled])

In [115]:
# Prepare the test dataset for prediction for Country B

In [116]:
b_h_test = treat_missing_values(b_h_test)
b_i_test = treat_missing_values(b_i_test)

In [117]:
b_h_test = normalize_dataset(b_h_test)
b_i_test = normalize_dataset(b_i_test)

In [118]:
b_h_test = create_feature_dummies(b_h_test)
b_i_test = create_feature_dummies(b_i_test)

In [119]:
from multiprocessing.pool import ThreadPool
pool = ThreadPool(processes=14)

final_test_df_b = pd.DataFrame()
results = []
start_time = time.time()
print("Time before starting the agregation operation for testing set B:",start_time)
for index in set(b_i_test.index.values):
    async_ret = pool.apply_async(aggregate_data, (b_i_test[b_i_test.index == index],index,"B")) # tuple of args for foo
    results.append(async_ret)

print('Scheduling complete... Waiting for results.')

for res in results:
    final_test_df_b = final_test_df_b.append(res.get())
    
print("Time after finishing the agregation operation for testing set B:",time.time()-start_time)
print('Run complete.')

Time before starting the agregation operation for testing set B: 1518969659.733398
Scheduling complete... Waiting for results.
Time after finishing the agregation operation for testing set B: 1433.3643889427185
Run complete.


In [120]:
final_test_df_b.drop_duplicates(inplace=True)
b_final_test_dataframe = pd.merge(final_test_df_b,b_h_test,left_index=True,right_index=True)

In [121]:
b_final_test_dataframe = pre_process(b_final_test_dataframe,enforce_cols=b_final_train_dataframe.columns)

In [122]:
# Initializing the Predictor and Target Variables for Modelling for Country B

In [123]:
from sklearn.model_selection import train_test_split
bX_train = b_final_train_dataframe.drop('poor',axis=1)
by_train = b_final_train_dataframe['poor']
#aX_train1,aX_test1,ay_train1,ay_test1 = train_test_split(aX_train,ay_train,test_size=0.20,random_state=243)

In [124]:
model_b = XGBClassifier()
#model_a.fit(aX_train1,ay_train1)
model_b.fit(bX_train,by_train)
b_final_test_dataframe = b_final_test_dataframe[bX_train.columns]
print(set(b_final_test_dataframe.columns.values) - set(bX_train.columns.values))
by_pred = model_b.predict_proba(b_final_test_dataframe)
by_pred

set()


array([[ 0.92961174,  0.07038826],
       [ 0.8064602 ,  0.19353981],
       [ 0.9345879 ,  0.06541212],
       ..., 
       [ 0.62731385,  0.37268612],
       [ 0.92933661,  0.07066338],
       [ 0.85807413,  0.14192589]], dtype=float32)

In [125]:
# Cleaning,Formatting and modelling for Country C

In [126]:
c_h_train = treat_missing_values(c_h_train)
c_i_train = treat_missing_values(c_i_train)

In [127]:
c_h_train = normalize_dataset(c_h_train)
c_i_train = normalize_dataset(c_i_train)

In [128]:
c_h_train = create_feature_dummies(c_h_train)
c_i_train = create_feature_dummies(c_i_train)

In [129]:
from multiprocessing.pool import ThreadPool
pool = ThreadPool(processes=14)

final_train_df_c = pd.DataFrame()
results = []
start_time = time.time()
print("Time before starting the agregation operation for training set C:",start_time)
for index in set(c_i_train.index.values):
    async_ret = pool.apply_async(aggregate_data, (c_i_train[c_i_train.index == index],index,"C")) # tuple of args for foo
    results.append(async_ret)

print('Scheduling complete... Waiting for results.')

for res in results:
    final_train_df_c = final_train_df_c.append(res.get())
print("Time after finishing the agregation operation for training set C:",time.time()-start_time)
print('Run complete.')

Time before starting the agregation operation for training set C: 1518971122.7042692
Scheduling complete... Waiting for results.
Time after finishing the agregation operation for training set C: 1179.1671957969666
Run complete.


In [130]:
# Remove duplicate rows from the individual falttened dataframe and concat it with household dataframe
final_train_df_c.drop_duplicates(inplace=True)
c_final_train_dataframe = pd.merge(final_train_df_c,c_h_train,left_index=True,right_index=True)
c_final_train_dataframe = c_final_train_dataframe.drop(['poor_y'],axis=1)
c_final_train_dataframe.rename(columns ={'poor_x':'poor'},inplace=True)
c_final_train_dataframe.head(n=5)

,OoqEwyJF_RuCZA,OoqEwyJF_cEcbt,cJPCnaAs_BZKME,cJPCnaAs_DMxNA,cJPCnaAs_Eadzw,cJPCnaAs_QhmHR,cJPCnaAs_UCneH,cJPCnaAs_ajgXa,cJPCnaAs_bvvXm,cJPCnaAs_cGrSo,cJPCnaAs_cMeuH,cJPCnaAs_uSuzR,vIUDQhEE_EThHI,vIUDQhEE_izIlz,vIUDQhEE_lJvCX,vIUDQhEE_zevDa,cRHfBOQd_GpRit,cRHfBOQd_HDYDE,cRHfBOQd_bCSuY,cRHfBOQd_sgjYl,dAmhsMgC_IpTiG,dAmhsMgC_gCSRj,dAmhsMgC_uEstx,skzSFBmO_WFHaK,skzSFBmO_oLMQK,skzSFBmO_zxTGu,IsqnlKAj_ALcKg,IsqnlKAj_FoQcU,IsqnlKAj_GpnOQ,IsqnlKAj_IkVYr,IsqnlKAj_JTCKs,IsqnlKAj_KRMzE,IsqnlKAj_OlCwh,IsqnlKAj_UaIsy,IsqnlKAj_VOLBH,IsqnlKAj_YNTOk,IsqnlKAj_aAJKs,IsqnlKAj_aNJHg,IsqnlKAj_dSJoN,IsqnlKAj_fUGkp,IsqnlKAj_kiHJe,IsqnlKAj_pVldP,IsqnlKAj_tIdUF,IsqnlKAj_vhhVz,IsqnlKAj_xqUoo,AOSWkWKB_XYcCK,AOSWkWKB_nLEWH,IThGOCux_AUbjP,IThGOCux_AnmxM,IThGOCux_BneRn,IThGOCux_BrMFM,IThGOCux_EGPlQ,IThGOCux_EYUZg,IThGOCux_EhzOz,IThGOCux_KKBwI,IThGOCux_LpkXG,IThGOCux_MyWVa,IThGOCux_PVIia,IThGOCux_QMwPt,IThGOCux_UrHEJ,IThGOCux_ZRSVD,IThGOCux_ehUOC,IThGOCux_gRXcL,IThGOCux_lJlxl,IThGOCux_mOVdm,IThGOCux_sVAQq,IThGOCux_wciCS,IThGOCux_zCkIA,gBTGbFhG_DfXNP,gBTGbFhG_JnveI,gBTGbFhG_KEvSa,gBTGbFhG_amOeQ,CggtnNTN_AdLvU,CggtnNTN_RxYsa,CggtnNTN_hAGot,CggtnNTN_xTmsI,TfEUOijE_AzTqj,TfEUOijE_BSfJh,TfEUOijE_CnPyM,TfEUOijE_FPaHY,TfEUOijE_ISjaW,TfEUOijE_Iwnmb,TfEUOijE_LXXwf,TfEUOijE_Lbazj,TfEUOijE_tPmhp,TfEUOijE_ucqiX,TfEUOijE_umeQR,TfEUOijE_zqVwN,tOfmAJyI_SSvEP,tOfmAJyI_TKOAj,tOfmAJyI_hCIpw,tOfmAJyI_mlNXN,FRcdTUFo_AOZvK,FRcdTUFo_BzCwH,FRcdTUFo_CfInb,FRcdTUFo_ChOam,FRcdTUFo_DnCsG,FRcdTUFo_EOMzW,FRcdTUFo_IkNZZ,FRcdTUFo_IqFnZ,FRcdTUFo_JOdpk,FRcdTUFo_Jarbl,FRcdTUFo_LlSiF,FRcdTUFo_NSXCO,FRcdTUFo_NSwMj,FRcdTUFo_NtAvB,FRcdTUFo_NwYCR,FRcdTUFo_Nyloj,FRcdTUFo_OjYmC,FRcdTUFo_OkwhI,FRcdTUFo_OoMwn,FRcdTUFo_PWYXC,FRcdTUFo_Pcfoh,FRcdTUFo_PwzSC,FRcdTUFo_QQdHS,FRcdTUFo_SPVgF,FRcdTUFo_UjTlg,FRcdTUFo_UveBM,FRcdTUFo_WaqYR,FRcdTUFo_WoXNm,FRcdTUFo_XjLjB,FRcdTUFo_YpOAj,FRcdTUFo_ZCuJD,FRcdTUFo_ZIrnY,FRcdTUFo_aYoNm,FRcdTUFo_aoyoS,FRcdTUFo_axSTs,FRcdTUFo_bSAHi,FRcdTUFo_efxMi,FRcdTUFo_exnST,FRcdTUFo_fgErF,FRcdTUFo_koaok,FRcdTUFo_mlwhh,FRcdTUFo_nQJlS,FRcdTUFo_nRNGN,FRcdTUFo_niWGM,FRcdTUFo_ocxWu,FRcdTUFo_poTXA,FRcdTUFo_qQkRd,FRcdTUFo_qqVib,FRcdTUFo_rAwvH,FRcdTUFo_rctWo,FRcdTUFo_ruoEr,FRcdTUFo_sMBUT,FRcdTUFo_ssIcX,FRcdTUFo_toWkd,FRcdTUFo_tzeVi,FRcdTUFo_ucHNS,FRcdTUFo_urrHd,FRcdTUFo_vcpPK,FRcdTUFo_wnEES,FRcdTUFo_xJeXb,FRcdTUFo_yWhop,FRcdTUFo_zIxnP,wXJsPzSl_EFUMy,wXJsPzSl_JskzT,wXJsPzSl_LPJYT,wXJsPzSl_cPXrX,EaHvfzfT_AYcgs,EaHvfzfT_GmSKW,EaHvfzfT_OiUlZ,EaHvfzfT_xHsSQ,EaHvfzfT_yFSGe,XDnwpOpR_DUVzU,XDnwpOpR_HuDgt,XDnwpOpR_nvyhU,XDnwpOpR_wsHHy,uOlSuJDG_HelFU,uOlSuJDG_UXvXi,uOlSuJDG_zJGmb,CjuMStGt_lYVyA,CjuMStGt_ukPfU,CjuMStGt_yAeCa,ymHYOZZv_Engma,ymHYOZZv_GHDuu,ymHYOZZv_JiqVt,ymHYOZZv_hOlGY,eXdISXQe_AmxrE,eXdISXQe_EHpdg,eXdISXQe_KhkzJ,eXdISXQe_MYsDv,eXdISXQe_UVolf,eXdISXQe_XkIZb,eXdISXQe_dxzZA,eXdISXQe_oLwml,eXdISXQe_uUUsm,eXdISXQe_wpqIb,WqmeSStq_DGWjH,WqmeSStq_XvXON,XDzQvdCw_PBUwt,XDzQvdCw_bgZsP,UTyHNliH_FbTRU,UTyHNliH_SOegK,UTyHNliH_XFrNG,UTyHNliH_hHSXB,UTyHNliH_xyraV,gxCmCVUk_EPnnG,gxCmCVUk_ZhnFe,gxCmCVUk_pClPr,gxCmCVUk_uSErG,gxCmCVUk_wNFGx,gxCmCVUk_yHvZA,gxCmCVUk_yVKQv,qGqYslGF_HXbvk,qGqYslGF_LuEXv,qGqYslGF_Qydia,qGqYslGF_RKJBq,qGqYslGF_SDNoA,qGqYslGF_UHDQy,qGqYslGF_bZAfO,qGqYslGF_hRHpW,qGqYslGF_kvMGu,qGqYslGF_oCaaP,qGqYslGF_rAwcc,qGqYslGF_sLhLU,qGqYslGF_vtkRP,dArQTUaf_BymTR,dArQTUaf_CcMHU,dArQTUaf_FSowp,dArQTUaf_FXwlB,dArQTUaf_FeIwW,dArQTUaf_Izoay,dArQTUaf_LAQpd,dArQTUaf_MrMfm,dArQTUaf_Pkbsl,dArQTUaf_RmtdH,dArQTUaf_SBtJM,dArQTUaf_UgMnz,dArQTUaf_VQjTo,dArQTUaf_VYpjn,dArQTUaf_VsHXJ,dArQTUaf_XFRBN,dArQTUaf_XQxHe,dArQTUaf_ZCotM,dArQTUaf_anwSJ,dArQTUaf_avKxn,dArQTUaf_dsFdu,dArQTUaf_eteYs,dArQTUaf_gvqxs,dArQTUaf_iQpoG,dArQTUaf_iRUzd,dArQTUaf_nDHku,dArQTUaf_pMMln,dArQTUaf_pWgvh,dArQTUaf_snUKz,dArQTUaf_vTJPA,dArQTUaf_wBmnI,NAxEQZVi_BgmGE,NAxEQZVi_GkrMH,NAxEQZVi_MZyJF,NAxEQZVi_Rihyc,ShCKQiAy_CRhHR,ShCKQiAy_GCqBR,ShCKQiAy_INYbJ,ShCKQiAy_QrjGn,ShCKQiAy_TYbsc,ShCKQiAy_YfuFP,ShCKQiAy_iuiyo,ShCKQiAy_mtKjA,ShCKQiAy_xJurw,rkLqZrQW_SoOdX,rkLqZrQW_jVHyH,rkLqZrQW_pbPGJ,rkLqZrQW_xUYIC,VGJlUgVG_AflBE,VGJlUgVG_FbszT,VGJlUgVG_

In [131]:
from sklearn.utils import resample

c_minority = c_final_train_dataframe[c_final_train_dataframe.poor == True]
c_majority = c_final_train_dataframe[c_final_train_dataframe.poor == False]
#Upsample minority class
c_minority_upsampled = resample(c_minority,replace=True,n_samples=5494,random_state=123)
c_final_train_dataframe = pd.concat([c_majority,c_minority_upsampled])

In [132]:
# Prepare the test dataset for prediction for Country C

In [133]:
c_h_test = treat_missing_values(c_h_test)
c_i_test = treat_missing_values(c_i_test)

In [134]:
c_h_test = normalize_dataset(c_h_test)
c_i_test = normalize_dataset(c_i_test)

In [135]:
c_h_test = create_feature_dummies(c_h_test)
c_i_test = create_feature_dummies(c_i_test)

In [136]:
from multiprocessing.pool import ThreadPool
pool = ThreadPool(processes=14)

final_test_df_c = pd.DataFrame()
results = []
start_time = time.time()
print("Time before starting the agregation operation for testing set C:",start_time)
for index in set(c_i_test.index.values):
    async_ret = pool.apply_async(aggregate_data, (c_i_test[c_i_test.index == index],index,"C")) # tuple of args for foo
    results.append(async_ret)

print('Scheduling complete... Waiting for results.')

for res in results:
    final_test_df_c = final_test_df_c.append(res.get())
print("Time after finishing the agregation operation for testing set C:",time.time()-start_time)
print('Run complete.')

Time before starting the agregation operation for testing set C: 1518972303.396234
Scheduling complete... Waiting for results.
Time after finishing the agregation operation for testing set C: 575.6526298522949
Run complete.


In [137]:
final_test_df_c.drop_duplicates(inplace=True)
c_final_test_dataframe = pd.merge(final_test_df_c,c_h_test,left_index=True,right_index=True)

In [138]:
c_final_test_dataframe = pre_process(c_final_test_dataframe,enforce_cols=c_final_train_dataframe.columns)

In [139]:
# Initializing the Predictor and Target Variables for Modelling for Country C

In [140]:
from sklearn.model_selection import train_test_split
cX_train = c_final_train_dataframe.drop('poor',axis=1)
cy_train = c_final_train_dataframe['poor']
#aX_train1,aX_test1,ay_train1,ay_test1 = train_test_split(aX_train,ay_train,test_size=0.20,random_state=243)

In [141]:
model_c = XGBClassifier()
#model_a.fit(aX_train1,ay_train1)
model_c.fit(cX_train,cy_train)
c_final_test_dataframe = c_final_test_dataframe[cX_train.columns]
print(set(c_final_test_dataframe.columns.values) - set(cX_train.columns.values))
cy_pred = model_c.predict_proba(c_final_test_dataframe)
cy_pred

set()


array([[  9.99514580e-01,   4.85444034e-04],
       [  7.08222389e-04,   9.99291778e-01],
       [  9.98739183e-01,   1.26082730e-03],
       ..., 
       [  3.60488892e-04,   9.99639511e-01],
       [  3.63230705e-04,   9.99636769e-01],
       [  9.99814689e-01,   1.85294994e-04]], dtype=float32)

In [142]:
print(len(ay_pred))
print(len(by_pred))
print(len(cy_pred))

4041
1604
3187


In [143]:
def make_country_sub(preds, test_feat, country):
    # make sure we code the country correctly
    country_codes = ['A', 'B', 'C']
    
    # get just the poor probabilities
    country_sub = pd.DataFrame(data=preds[:, 1],  # proba p=1
                               columns=['poor'], 
                               index=test_feat.index)

    
    # add the country code for joining later
    country_sub["country"] = country
    return country_sub[["country", "poor"]]

In [144]:
# convert preds to data frames
a_sub = make_country_sub(ay_pred, a_h_test, 'A')
b_sub = make_country_sub(by_pred, b_h_test, 'B')
c_sub = make_country_sub(cy_pred, c_h_test, 'C')

In [145]:
submission_test2 = pd.concat([a_sub, b_sub, c_sub])

In [146]:
submission_test2.to_csv('submission_final.csv')